In [39]:
# insert matplotlib in colab
%matplotlib inline

In [40]:
import torch
from torch import  nn
from torch.utils.data import  DataLoader
# als many domian specific lib,like torchaudio, torchtext
from torchvision import datasets
# datasets: CIFAR, COCO, etc
# also torchvision.datasets include 2 argument: transform, taget_transform to modify the samples
from torchvision.transforms import ToTensor

In [41]:
#Dowload training data from open datasets
#!datasets.FashionMNIST?
training_data = datasets.FashionMNIST(
   root = "data",
   train = True,
   download = True,
   transform = ToTensor()
)

test_data = datasets.FashionMNIST(
   root = "data",
   train = False,
   download = True,
   transform = ToTensor()
)

In [42]:
#pass the data to Dataloader
#Dataloader wraps an iterable over our dataset, and support automatic batching, sampling, shulffing and multiprocessing data loading
#!DataLoader?
#batch size is number of samples processed before model updated
#small batch size, need more epoched to convergent, large may miss the local lowest point.
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(dataset=training_data, batch_size=batch_size)
test_dataloader = DataLoader(dataset=test_data, batch_size=batch_size)

""" for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break """

' for X, y in test_dataloader:\n    print(f"Shape of X [N, C, H, W]: {X.shape}")\n    print(f"Shape of y: {y.shape} {y.dtype}")\n    break '

In [43]:
# Get cpu or gpu device for training.
# mps: meta performance shaders for macos with Apple silicon
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

# Define model
# Inherits from nn.Module
# nn.Module: base class for all nn module, your model should also subclass this class, nest them in a tree structure
class NeuralNetwork(nn.Module):
    #define the layers in NN
    def __init__(self):
        #super() with __init__()
        super().__init__()
        self.flatten = nn.Flatten()
        #logits
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    #define how the data pass the model
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [44]:
# loss function
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(),  lr=1e-3)

In [45]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [46]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [47]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.315323  [    0/60000]
loss: 2.295849  [ 6400/60000]
loss: 2.279877  [12800/60000]
loss: 2.263574  [19200/60000]
loss: 2.240759  [25600/60000]
loss: 2.236761  [32000/60000]
loss: 2.222672  [38400/60000]
loss: 2.201022  [44800/60000]
